In [104]:
from web3 import Web3
import requests
import pandas as pd
import json

## setup the eth node (if infura plug in project id)
dfweb3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/d84606165772420ea3b28a05ff5b0d97'))

## query airtable for data
airtable_api = 'key8uWPrOLIaKd8ri'  # update from testing API
baseid = 'app5ZlqY2U36bO2oK'

tableid_keys = 'tbl2Vm1bpHmNOEafe'
tableid_eth = 'tbll8IkER4g9sUPzf'
url_keys = f"https://api.airtable.com/v0/{baseid}/{tableid_keys}"
url_eth = f"https://api.airtable.com/v0/{baseid}/{tableid_eth}"

headers = {'Authorization': f'Bearer {airtable_api}'}

response_eth = requests.request("GET", url_eth, headers=headers).json()['records']
response_keys = requests.request("GET", url_keys, headers=headers).json()['records']

# turn response into df
airtable_rows = [] 
airtable_index = []
for record in response_eth:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df_eth = pd.DataFrame(airtable_rows, index=airtable_index)

airtable_rows = [] 
airtable_index = []
for record in response_keys:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df_keys = pd.DataFrame(airtable_rows, index=airtable_index)

In [82]:
# to have less probability of same FromWallet getting hit in a row + bigger transactions first
df_eth = df_eth.sort_values(by="ethAmount", ascending=False).sort_values(by="ToWalletAddress")

In [105]:
df_eth

,ethAmount,ToWalletAddress,FromWalletAddress,Status
rec3qO82V3LJXh2HP,0.002,0x6796f099A019CC6084D3B52284D40823fc09D730,0x3333333,NaN
recMLjvT7GS9lKdmg,0.001,0x6796f099A019CC6084D3B52284D40823fc09D730,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,NaN
recPP70qf0ttUeAeN,0.002,0x6796f099A019CC6084D3B52284D40823fc09D730,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,NaN
recqRn45VwjlMXkW8,300,0x6796f099A019CC6084D3B52284D40823fc09D730,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,FAILED


In [84]:
df_keys

,WalletAddress,PrivateKey
recSO9JPrp7d6bRqK,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,597b5a3feb884ca6f69cf7d84999999999
rectbmPugBPrGoBlB,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,597b5a3feb884ca6f69cf7d8496a1c62acba5fa8692f25...


In [66]:
# Running available transactions first and marking those blocked as "failed" to be checked later
for index, row in df_eth.iterrows():    
    eth = row["ethAmount"]
    to_w = row["ToWalletAddress"]
    from_w = row["FromWalletAddress"]
    nonce = web3.eth.getTransactionCount(from_w)
    
    try:
        #locate private key
        private_key = ''
        for index, row in df_keys.iterrows():
            if row['WalletAddress'] == from_w:
                private_key = row['PrivateKey']
            else:
                continue
        
        ## get gas and gas price
        gas_price = web3.eth.gasPrice / 10**9 # turn into gwei
        gas = web3.eth.estimateGas({
            "from": from_w,
            "nonce": nonce,
            "to": to_w,
            "value": 2  # convert to 'round(amount)' for more dynamic gas
            })

        #build a transaction in a dictionary
        tx = {
            'nonce': nonce,
            'to': to_w,
            'value': web3.toWei(eth, 'ether'),  # Ether can be split into 10^18 Wei
            'gas': gas,  # amount of gas
            'gasPrice': web3.toWei(gas_price, 'gwei')  # gas price updated
            }

        #sign the transaction with private key
        signed_tx = web3.eth.account.sign_transaction(tx, private_key)

        #send transaction to the eth node
        tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)

        #get transaction hash
        print("Completed " + web3.toHex(tx_hash) + " tx for " + eth + "eth")
        
        #PATCH into airtable the hash of successful transaction into status column
        record_id = index
        record_url = url_eth + "/" + record_id
        headers = {'Authorization': f'Bearer {airtable_api}', "Content-type": "application/json"}

        upload_data = {"Status" : tx_hash}
        upload_dict = {"fields" : upload_data}
        upload_json = json.dumps(upload_dict)
        response_patch = requests.patch(record_url, data=upload_json, headers=headers)
        
    except: 
        #PATCH into airtable a "FAILED" into status column
        record_id = index
        record_url = url_eth + "/" + record_id
        headers = {'Authorization': f'Bearer {airtable_api}', "Content-type": "application/json"}

        upload_data = {"Status" : "FAILED"}
        upload_dict = {"fields" : upload_data}
        upload_json = json.dumps(upload_dict)
        response_patch = requests.patch(record_url, data=upload_json, headers=headers)

300
0.002
0.002
0.001


In [86]:
for index, row in df_eth.iterrows():
    print(index)

recqRn45VwjlMXkW8
rec3qO82V3LJXh2HP
recPP70qf0ttUeAeN
recMLjvT7GS9lKdmg


In [103]:
import json
# Patch testing
# URL needs to point towards the specific record we are updating
url_eth = f"https://api.airtable.com/v0/{baseid}/{tableid_eth}"
record_id = "recqRn45VwjlMXkW8"
record_url = url_eth + "/" + record_id
headers = {'Authorization': f'Bearer {airtable_api}', "Content-type": "application/json"}

upload_data = {"Status" : "FAILED"}
upload_dict = {"fields" : upload_data}
upload_json = json.dumps(upload_dict)
response_patch = requests.patch(record_url, data=upload_json, headers=headers)
print(upload_dict)
response_patch.json()

{'fields': {'Status': 'FAILED'}}


{'id': 'recqRn45VwjlMXkW8',
 'createdTime': '2022-10-11T14:20:11.000Z',
 'fields': {'Status': 'FAILED',
  'ethAmount': '300',
  'ToWalletAddress': '0x6796f099A019CC6084D3B52284D40823fc09D730',
  'FromWalletAddress': '0x344ED1Bac25fdB1d5F4079790AE486FEc524580a'}}

In [88]:
df_eth.dtypes

ethAmount            object
ToWalletAddress      object
FromWalletAddress    object
dtype: object

In [108]:
df_eth["ToWalletAddress"] == "0x6796f099A019CC6084D3B52284D40823fc09D730"

rec3qO82V3LJXh2HP    True
recMLjvT7GS9lKdmg    True
recPP70qf0ttUeAeN    True
recqRn45VwjlMXkW8    True
Name: ToWalletAddress, dtype: bool

In [109]:
df_eth[df_eth["ToWalletAddress"] == "0x6796f099A019CC6084D3B52284D40823fc09D730"]

,ethAmount,ToWalletAddress,FromWalletAddress,Status
rec3qO82V3LJXh2HP,0.002,0x6796f099A019CC6084D3B52284D40823fc09D730,0x3333333,NaN
recMLjvT7GS9lKdmg,0.001,0x6796f099A019CC6084D3B52284D40823fc09D730,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,NaN
recPP70qf0ttUeAeN,0.002,0x6796f099A019CC6084D3B52284D40823fc09D730,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,NaN
recqRn45VwjlMXkW8,300,0x6796f099A019CC6084D3B52284D40823fc09D730,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,FAILED


In [106]:
df_eth[df_eth["Status"] == "FAILED"]

,ethAmount,ToWalletAddress,FromWalletAddress,Status
recqRn45VwjlMXkW8,300,0x6796f099A019CC6084D3B52284D40823fc09D730,0x344ED1Bac25fdB1d5F4079790AE486FEc524580a,FAILED


In [118]:
# Testing Etherscan API
# Returns the Ether balance of a given address.
from_w = "0x344ED1Bac25fdB1d5F4079790AE486FEc524580a"
url_escan = "https://api.etherscan.io/api"
escan_api = "KI6E261EI7Z12KNUIAY35S7JMYAC8EMD1B"

# eth response in wei
params = {"module":"account", "action":"balance", "address":from_w, "tag":"latest", "apikey":escan_api}

int(requests.get(url_escan, params=params).json()['result'])/ 10**18

0.5190084034894882

In [107]:
# Running through "FAILED" transaction and checking ETHERSCAN for issues

# Pandas filter df_eth for failed
df_failed = df_eth[df_eth["Status"] == "FAILED"]

for index, row in df_failed.iterrows():
    eth = row["ethAmount"]
    to_w = row["ToWalletAddress"]
    from_w = row["FromWalletAddress"]
    nonce = web3.eth.getTransactionCount(from_w)
    
    try:
        #locate private key 
        private_key = ''
        for index, row in df_keys.iterrows():
            if row['WalletAddress'] == from_w:
                private_key = row['PrivateKey']
            else:
                continue
        
        ## get gas and gas price
        gas_price = web3.eth.gasPrice / 10**9 # turn into gwei
        gas = web3.eth.estimateGas({
            "from": from_w,
            "nonce": nonce,
            "to": to_w,
            "value": 2  # convert to 'round(amount)' for more dynamic gas
            })

        #build a transaction in a dictionary
        tx = {
            'nonce': nonce,
            'to': to_w,
            'value': web3.toWei(eth, 'ether'),  # Ether can be split into 10^18 Wei
            'gas': gas,  # amount of gas
            'gasPrice': web3.toWei(gas_price, 'gwei')  # gas price updated
            }

        #sign the transaction with private key
        signed_tx = web3.eth.account.sign_transaction(tx, private_key)

        #send transaction to the eth node
        tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)

        #get transaction hash
        print("Completed " + web3.toHex(tx_hash) + " tx for " + eth + "eth")
        
        #PATCH into airtable the hash of successful transaction into status column
        record_id = index
        record_url = url_eth + "/" + record_id
        headers = {'Authorization': f'Bearer {airtable_api}', "Content-type": "application/json"}

        upload_data = {"Status" : tx_hash}
        upload_dict = {"fields" : upload_data}
        upload_json = json.dumps(upload_dict)
        response_patch = requests.patch(record_url, data=upload_json, headers=headers)
        
    except:  
        # Check if wallet funds are too low for transaction + patch that airtable
        url_escan = "https://api.etherscan.io/api"
        escan_api = "KI6E261EI7Z12KNUIAY35S7JMYAC8EMD1B"
        params = {"module":"account", "action":"balance", "address":from_w, "tag":"latest", "apikey":escan_api}
        funds = int(requests.get(url_escan, params=params).json()['result'])/ 10**18 # eth response in wei
        if funds < eth:
            #PATCH into "Failed" column the lack of funds
            record_id = index
            record_url = url_eth + "/" + record_id
            headers = {'Authorization': f'Bearer {airtable_api}', "Content-type": "application/json"}

            upload_data = {"Status" : "LOW FUNDS"}
            upload_dict = {"fields" : upload_data}
            upload_json = json.dumps(upload_dict)
            response_patch = requests.patch(record_url, data=upload_json, headers=headers)
            
        # Next reason should be a pending transaction so make code wait 1m and try again transaction
        

SyntaxError: unexpected EOF while parsing (1806442427.py, line 56)

In [ ]:
import time

while True:
  localtime = time.localtime()
  result = time.strftime("%I:%M:%S %p", localtime)
  print(result)
  time.sleep(1)

In [119]:
# TEST
# Create new accounts on metamask and send them fake eth https://faucet.paradigm.xyz/
# Need to check etherscan test network also (how to query it)